In [ ]:
import csv
import numpy as np
import random

In [133]:
file = open("new_train.csv", "r", encoding='utf-8')
data = list(csv.reader(file, delimiter=","))
file.close()
data = data[1:]
data[0]

['0',
 ' Emergency Room Reports',
 'REASON FOR THE VISIT:,  Very high PT/INR.,HISTORY: , The patient is an 81-year-old lady whom I met last month when she came in with pneumonia and CHF.  She was noticed to be in atrial fibrillation, which is a chronic problem for her.  She did not want to have Coumadin started because she said that she has had it before and the INR has had been very difficult to regulate to the point that it was dangerous, but I convinced her to restart the Coumadin again.  I gave her the Coumadin as an outpatient and then the INR was found to be 12.  So, I told her to come to the emergency room to get vitamin K to reverse the anticoagulation.,PAST MEDICAL HISTORY:,1.  Congestive heart failure.,2.  Renal insufficiency.,3.  Coronary artery disease.,4.  Atrial fibrillation.,5.  COPD.,6.  Recent pneumonia.,7.  Bladder cancer.,8.  History of ruptured colon.,9.  Myocardial infarction.,10.  Hernia repair.,11.  Colon resection.,12.  Carpal tunnel repair.,13.  Knee surgery.,M

In [132]:
# Get a map from label numbers to names
label_map = {}
for sample in data:
    if int(sample[-1]) not in label_map:
        label_map[int(sample[-1])] = sample[1].strip()
print(label_map)

{0: 'Emergency Room Reports', 1: 'Surgery', 2: 'Radiology', 3: 'Podiatry', 4: 'Neurology', 5: 'Gastroenterology', 6: 'Orthopedic', 7: 'Cardiovascular / Pulmonary', 8: 'Nephrology', 9: 'ENT - Otolaryngology', 10: 'General Medicine', 11: 'Hematology - Oncology', 12: 'Cosmetic / Plastic Surgery', 13: 'SOAP / Chart / Progress Notes', 14: 'Chiropractic', 15: 'Psychiatry / Psychology', 16: 'Consult - History and Phy.', 17: 'Hospice - Palliative Care', 18: 'Neurosurgery', 19: 'Obstetrics / Gynecology', 20: 'Urology', 21: 'Discharge Summary', 22: 'Autopsy', 23: 'Dermatology', 24: 'Letters', 25: 'Office Notes', 26: 'Lab Medicine - Pathology', 27: 'Ophthalmology', 28: 'Speech - Language', 29: 'Dentistry', 30: 'Pediatrics - Neonatal', 31: 'Physical Medicine - Rehab', 32: 'Bariatrics', 33: 'Endocrinology', 34: 'Pain Management', 35: 'IME-QME-Work Comp etc.', 36: 'Allergy / Immunology', 37: 'Sleep Medicine', 38: 'Diets and Nutritions', 39: 'Rheumatology'}


In [136]:
# Find frequency of each label in training data
def get_probabilities(data):
    counts = {}
    for sample in data:
        if int(sample[-1]) not in counts:
            counts[int(sample[-1])] = 0
        counts[int(sample[-1])] += 1
    s = sum(counts.values())
    probabilities = counts
    for k in probabilities:
        probabilities[k] /= s
    return probabilities
get_probabilities(data)

{9: 0.020660115898211137,
 5: 0.044343663391282435,
 16: 0.10330057949105569,
 19: 0.030990173847316706,
 4: 0.04283194759385236,
 1: 0.21743512219702696,
 11: 0.017132779037540943,
 2: 0.05366591080876795,
 10: 0.052658100277147894,
 30: 0.013857394809775762,
 7: 0.07785336356764928,
 18: 0.017888636936255985,
 21: 0.021919879062736205,
 6: 0.072814310909549,
 14: 0.0030234315948601664,
 37: 0.0030234315948601664,
 0: 0.015873015873015872,
 29: 0.005291005291005291,
 20: 0.033761652809271854,
 24: 0.004787100025195263,
 34: 0.013605442176870748,
 13: 0.034013605442176874,
 8: 0.015873015873015872,
 33: 0.004031242126480222,
 25: 0.009574200050390527,
 23: 0.005291005291005291,
 27: 0.016880826404635927,
 32: 0.003779289493575208,
 28: 0.002015621063240111,
 15: 0.011337868480725623,
 38: 0.0022675736961451248,
 3: 0.008818342151675485,
 12: 0.004787100025195263,
 26: 0.0012597631645250692,
 31: 0.004031242126480222,
 35: 0.0030234315948601664,
 17: 0.0015117157974300832,
 22: 0.001763

In [ ]:
def tokenize(s):
    '''Filter out non-alphabet characters
    Standardize case
    Return list of words'''
    out = []
    cur = ""
    for c in s:
        if c.isalpha():
            cur += c.lower()
        elif len(cur) > 0:
            out.append(cur)
            cur = ""
    return out

In [ ]:
# Get full list of words that appear
def get_word_list(data):
    words = set()
    for sample in data:
        for word in tokenize(sample[2]):
            # Filter out common words
            if len(word) > 2:
                words.add(word)
    return words
word_list = get_word_list(data)

In [ ]:
# Get frequency of words in data set
def get_word_frequencies(data):
    word_frequencies = {}
    for sample in data:
        words = set(tokenize(sample[2]))
        for word in words:
            if word not in word_frequencies:
                word_frequencies[word] = 0
            word_frequencies[word] += 1
    s = sum(word_frequencies.values())
    for word in word_frequencies:
        word_frequencies[word] /= s
    return word_frequencies

In [ ]:
# Create a look-up table for the probability of a sample being a given class
# given a word w is in the sample
def create_lookup_table(word_list, data):
    table = {}
    sample_sets = []
    for sample in data:
        word_set = set(tokenize(sample[2]))
        k = int(sample[3])  # class
        for word in word_set:
            if word not in word_list:
                continue
            if word not in table:
                table[word] = {}
            if k not in table[word]:
                table[word][k] = 0
            table[word][k] += 1
    for word in table:
        s = sum(table[word].values())
        for k in table[word]:
            table[word][k] /= s

    return table

prob_table = create_lookup_table(word_list, data)

In [ ]:
def naive_bayes(text, probabilities, prob_table, word_frequencies):
    words = set(tokenize(text))
    best_class = -1
    best_prob = 0
    for label in label_map:
        if label not in probabilities:
            continue
        cur_prob = np.log(probabilities[label])
        for word in words:
            if word in prob_table and word in word_frequencies:
                if label in prob_table[word]:
                    cur_prob += np.log(prob_table[word][label])
                    #print(word, prob_table[word][label] / word_frequencies[word], prob_table[word][label], word_frequencies[word], cur_prob)
                else:
                    #print(label, word)
                    cur_prob = 1
            if cur_prob == 1:
                break
        #print(label, cur_prob)
        if 1 > cur_prob and (cur_prob > best_prob or best_class == -1):
            best_class = label
            best_prob = cur_prob
    return best_class, best_prob


In [138]:
# Test model
random.shuffle(data)
segments = []
for i in range(10):
    segments.append(data[(i*len(data))//10:((i+1)*len(data))//10])
for i in range(10):
    train = []
    for j in range(10):
        if j != i:
            train.extend(segments[i])
    test = segments[i]
    probabilities = get_probabilities(train)
    word_list = get_word_list(train)
    prob_table = create_lookup_table(word_list, train)
    word_frequencies = get_word_frequencies(train)
    correct = 0
    preds = []
    ts = []
    for sample in test:
        pred, _ = naive_bayes(sample[2], probabilities, prob_table, word_frequencies)
        if pred == int(sample[3]):
            correct += 1
        preds.append(pred)
        ts.append(int(sample[3]))
    # Calculate F1 score
    f1 = 0
    for j in label_map:
        f_neg = 0
        pos = 0
        p_pos = 0
        t_pos = 0
        for pred, t in zip(preds, ts):
            if pred == j:
                p_pos += 1
            if t == j:
                pos += 1
                if pred != j:
                    f_neg += 1
                else:
                    t_pos += 1
        if t_pos > 0:
            f1 += 2 / (p_pos / t_pos + pos / t_pos)
    f1 /= len(label_map)
    print(i, f1)


0 0.7505102754780697
1 0.8098715742889556
2 0.8719104759515108
3 0.863483051378766
4 0.830973844190232
5 0.7783300353938858
6 0.7119673615211669
7 0.8174002823881661
8 0.7775076832763947
9 0.7262653135642881


In [135]:
print(list(zip(preds, t)))

[(9, 9, -541.8803549657874), (5, 5, -488.6353880631637), (16, 16, -372.9608439596602), (19, 19, -341.0475813472801), (4, 4, -838.4421015884645), (1, 1, -319.4247897299166), (11, 11, -787.9327196130979), (2, 2, -202.9612179870478), (2, 2, -234.56382068551895), (16, 16, -277.12275781802845), (10, 10, -681.1572461658214), (4, 4, -124.75581020065268), (16, 16, -700.1476789415799), (5, 5, -243.00218909048073), (5, 5, -874.3141606852299), (10, 10, -540.085169905889), (1, 1, -416.87903448871714), (30, 30, -816.4555514111679), (16, 16, -123.81054160441641), (11, 11, -719.0726605718099), (7, 7, -401.0275345352372), (5, 5, -434.476472605986), (18, 18, -416.840088635203), (1, 1, -189.0940441471926), (5, 5, -294.70210713060175), (21, 21, -168.3620486473737), (6, 6, -215.6378565959888), (16, 16, -1116.0353722564362), (1, 1, -127.1794620295279), (16, 16, -590.1947171320218), (14, 14, -1522.8190064885084), (13, 37, -1035.843456205321), (10, 0, -447.52396103993067), (5, 5, -330.7507027095444), (2, 2, 

In [ ]:
print(probabilities)

In [ ]:
naive_bayes(data[1][2], probabilities, prob_table, word_frequencies)

In [ ]:
label_map

In [ ]:
word_frequencies